In [4]:
from models import *
from strategies import *
from datasets import *
import numpy as np

import tqdm
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
import os
import torch
import torch.nn as nn

from PIL import Image
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset

In [5]:
params = {'n_epoch': 50, 
               'train_args':{'batch_size': 1, 'num_workers': 0},
               'test_args':{'batch_size': 1, 'num_workers': 0},
               'optimizer_args':{'lr': 0.00005, 'momentum': 0.5}}

In [6]:
def get_chorus_data(handler):
    raw_train = Chorus_dataset("processed/", split='train', init=True, transform=True)
    raw_test = Chorus_dataset("processed/", split='test', init=True, transform=True)
    return Data(raw_train.images, raw_train.labels, raw_test.images, raw_test.labels, handler)


In [7]:

model = smp.create_model(
            'FPN', encoder_name='resnet34', in_channels=3, classes = 1
        )
net = Net(model, params, device = torch.device("cuda:0"))
data = get_chorus_data(Handler)
data.initialize_labels(5)



### Choose an AL strategy from a)RandomSampling b)MarginSampling c)EntropySampling d)KCenterGreedy e)AdversarialBIM

In [8]:
strategy = MarginSampling(data, net)


In [ ]:
print("Round 0")
strategy.train()
logits, mask_gt = strategy.predict(data.get_test_data())
print(f"Round 0 testing metrics: {data.cal_test_metrics(logits, mask_gt )}")

for rd in range(1, 13):
    print(f"Round {rd}")

    # query
    query_idxs = strategy.query(5)

    # update labels
    strategy.update(query_idxs)
    strategy.train()

    # calculate accuracy
    logits, maks_gt = strategy.predict(data.get_test_data())
    print(f"Round {rd} testing metrics: {data.cal_test_metrics(logits, mask_gt)}")

Round 0


 98%|█████████████████████████████████████████████████████████████▋ | 49/50 [00:30<00:00,  1.69it/s]